In [9]:
my ={}
my['test'] =9
if my:
    print(my)
else:
    print('empty')

{'test': 9}


In [1]:
import ezdxf
from dxfgrouping import filterlayerelements, makemergevertset, linearizeelement
from dxfgrouping import getblockcomponent, dxfoutputblocks

def convertall(inputfilename, outputfilename, cutlayers, penlayers, breflY, dmax):
    d = ezdxf.readfile(inputfilename)
    print("layerspresent:", set(e.dxf.layer for e in d.entities))
    d = ezdxf.readfile(inputfilename)
    lwpolylines = [ e  for e in d.entities  if e.dxftype() == "LWPOLYLINE" ]
    if lwpolylines:
        #print("exploding %d lwpolylines" % len(lwpolylines))
        for e in lwpolylines:
            e.explode()

    cutelements = filterlayerelements(d, cutlayers)
    penelements = filterlayerelements(d, penlayers)
        
    lcutelements, lpenelements = cutelements, penelements
    elementgroups = [ ]

    while lcutelements:
        res = getblockcomponent(lcutelements, lpenelements, dmax)
        (outercutelements, outercutelementsdir) = res[0]
        (internalcutelements, internalpenelements) = res[1]
        (remainingcutelements, remainingpenelements) = res[2]
        elementgroups.append((outercutelements, outercutelementsdir, internalcutelements, internalpenelements))
        lcutelements, lpenelements = remainingcutelements, remainingpenelements
        #print("Component: ", len(outercutelements), len(internalcutelements), len(internalpenelements))
        
    dxfoutputblocks(outputfilename, elementgroups, breflY)
    return elementgroups


In [2]:
import os
def createorclear(aamaoutdir, lsubdir):
    subdir = os.path.join(aamaoutdir, lsubdir)
    if os.path.exists(subdir):
        for f in os.listdir(subdir):
            os.remove(os.path.join(subdir, f))
    else:
        os.mkdir(subdir)
    return subdir
        


In [3]:
#-F15-BS-01- Bottom surface. 
#  BS-CUT layer to be cut, BS-PLOT to be plotted, all pieces cut from '170 color'

# All need left and right hand versions.

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2

inputfilename = "../dxffiles/F15-BS-01.dxf"
d170color = createorclear(aamaoutdir, "170color")
cutlayers, penlayers = ["BS-CUT"], ['BS-PLOT']
outputfilename = os.path.join(d170color, "F15-BS.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d170color, "F15-BS-reflY.dxf")
k = convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)


layerspresent: {'BS-PLOT', 'REFERENCE', 'CONSTRUCTION', 'HISTORY', 'BS-CUT'}
dmaxs-tail:  [6.245036057586848e-12, 1.0999348430804918e-11, 0.06873168711912457]
layerspresent: {'BS-PLOT', 'REFERENCE', 'CONSTRUCTION', 'HISTORY', 'BS-CUT'}
dmaxs-tail:  [6.245036057586848e-12, 1.0999348430804918e-11, 0.06873168711912457]


In [5]:
from dxfgrouping import cutcontouraspoly, dxfoutputblocks
ptsseq = cutcontouraspoly(k[0][0], k[0][1])
contourcentre = ezdxf.math.vector.Vector((min(p.x  for p in ptsseq) + max(p.x  for p in ptsseq))/2, (min(p.y  for p in ptsseq) + max(p.y  for p in ptsseq))/2, 0)
contourcentre

Vector(2800.27996743414, -229.83702825258047, 0.0)

In [6]:
#-F15-TSF-Master-04.dxf. Top surface front (leading edge). 
#  TSF-CUT and TSF-PLOT to be cut from '210 mast sleeve' 
#  PATCH-NORM CUT and PLOT to be from 210 mast sleeve if making a standard glider
#    or PATCH-CARR_CUT and PLOT to be from 210 mast sleeve if making a carrier wing one. 
#  MYLAR-CUT to cut from Mylar.

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2

carrierwingglider = True

inputfilename = "../dxffiles/F15-TSF-Master-05.dxf"
d210mastsleeve = createorclear(aamaoutdir, "210mastsleeve")
cutlayers, penlayers = ['TSF-CUT'], ['TSF-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-TSF.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if carrierwingglider:
    cutlayers, penlayers = ['PATCH-CARR-CUT'], ['PATCH-CARR-PLOT']
else:
    cutlayers, penlayers = ['PATCH-NORM-CUT'], ['PATCH-NORM-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

cutlayers, penlayers = ['MYLAR-CUT'], []
dmylar = createorclear(aamaoutdir, "mylar")
outputfilename = os.path.join(dmylar, "F15-TSF-MYLAR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(dmylar, "F15-TSF-MYLAR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)
;

layerspresent: {'TSF-PLOT', 'PATCH-REF', 'PATCH-NORM-CUT', 'TSF-CUT', 'CONSTRUCTION', 'MYLAR-CUT', 'PATCH-CARR-PLOT', 'PATCH-NORM-PLOT', 'HISTORY', 'PATCH-CARR-CUT'}
layerspresent: {'TSF-PLOT', 'PATCH-REF', 'PATCH-NORM-CUT', 'TSF-CUT', 'CONSTRUCTION', 'MYLAR-CUT', 'PATCH-CARR-PLOT', 'PATCH-NORM-PLOT', 'HISTORY', 'PATCH-CARR-CUT'}
layerspresent: {'TSF-PLOT', 'PATCH-REF', 'PATCH-NORM-CUT', 'TSF-CUT', 'CONSTRUCTION', 'MYLAR-CUT', 'PATCH-CARR-PLOT', 'PATCH-NORM-PLOT', 'HISTORY', 'PATCH-CARR-CUT'}
dmaxs-tail:  [0.0002619047815896499, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [0.0002619047815896499, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [0.0002619047815896499, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [0.0002619047815896499, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [0.0002619047815896499, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [9.094947017729282e-13, 0.044846696606814195, 0.05489101224166145]
dmaxs-tail:  [0.0, 9.0

''

In [8]:
#-F15-TSR-TSM-Master-04.dxf. Top surface middle and top surface rear panels. 
#  TSR and TSM cut and plot from either 137cm wide Diax, 144cm wide 170 white or color or 150cm wide ODL06. 
#  Patch cut and patch-plot from 170 color (same as BS).

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2
b144type = False
 
carrierwingglider = False

inputfilename = "../dxffiles/F15-TSR-TSM-Master-04.dxf"
ddiax = createorclear(aamaoutdir, "ODL06")



if b144type:
    cutlayers, penlayers = ['TSR-144-CUT'], ['TSR-144-PLOT']
else:
    cutlayers, penlayers = ['TSR-150-CUT'], ['TSR-150-PLOT']
outputfilename = os.path.join(ddiax, "F15-TSR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "F15-TSR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if b144type:
    cutlayers, penlayers = ['TSM-144-CUT'], ['TSM-144-PLOT']
else:
    cutlayers, penlayers = ['TSM-150-CUT'], ['TSM-150-PLOT']
outputfilename = os.path.join(ddiax, "F15-TSM.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "F15-TSM-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

print("writing to", d210mastsleeve)
cutlayers, penlayers = ['PATCH-CUT'], ['PATCH-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d210mastsleeve, "F15-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)
;

layerspresent: {'PATCH-PLOT', '0', 'PATCH-CUT', 'TSM-144-PLOT', 'TSR-144-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSR-150-CUT', 'REFERENCE', 'CONSTRUCTION', 'TSR-144-CUT', 'TSR-150-PLOT', 'HISTORY', 'TSM-150-PLOT'}
dmaxs-tail:  [1.3191514447337417e-10, 0.0020259419136499047, 0.15062684391013764]
layerspresent: {'PATCH-PLOT', '0', 'PATCH-CUT', 'TSM-144-PLOT', 'TSR-144-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSR-150-CUT', 'REFERENCE', 'CONSTRUCTION', 'TSR-144-CUT', 'TSR-150-PLOT', 'HISTORY', 'TSM-150-PLOT'}
dmaxs-tail:  [1.3191514447337417e-10, 0.0020259419136499047, 0.15062684391013764]
layerspresent: {'PATCH-PLOT', '0', 'PATCH-CUT', 'TSM-144-PLOT', 'TSR-144-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSR-150-CUT', 'REFERENCE', 'CONSTRUCTION', 'TSR-144-CUT', 'TSR-150-PLOT', 'HISTORY', 'TSM-150-PLOT'}
layerspresent: {'PATCH-PLOT', '0', 'PATCH-CUT', 'TSM-144-PLOT', 'TSR-144-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSR-150-CUT', 'REFERENCE', 'CONSTRUCTION', 'TSR-144-CUT', 'TSR-150-PLOT', 'HISTORY', 'TSM

''

In [9]:
#Fly15 Keel pocket, no reflection needed

inputfilename = "../dxffiles/F15-KP-01.dxf"
cutlayers, penlayers = ["KP-CUT"], ['KP-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-KP.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

layerspresent: {'KP-CUT', 'KP-PLOT'}


[([<class 'ezdxf.entities.arc.Arc'> ARC(#55),
   <class 'ezdxf.entities.line.Line'> LINE(#54),
   <class 'ezdxf.entities.arc.Arc'> ARC(#53),
   <class 'ezdxf.entities.line.Line'> LINE(#52),
   <class 'ezdxf.entities.arc.Arc'> ARC(#51),
   <class 'ezdxf.entities.line.Line'> LINE(#50),
   <class 'ezdxf.entities.arc.Arc'> ARC(#4F),
   <class 'ezdxf.entities.line.Line'> LINE(#67),
   <class 'ezdxf.entities.line.Line'> LINE(#66),
   <class 'ezdxf.entities.line.Line'> LINE(#68),
   <class 'ezdxf.entities.line.Line'> LINE(#62),
   <class 'ezdxf.entities.arc.Arc'> ARC(#59),
   <class 'ezdxf.entities.line.Line'> LINE(#5A),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5B),
   <class 'ezdxf.entities.line.Line'> LINE(#5C),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5D),
   <class 'ezdxf.entities.line.Line'> LINE(#5E),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5F),
   <class 'ezdxf.entities.line.Line'> LINE(#60),
   <class 'ezdxf.entities.arc.Arc'> ARC(#61),
   <class 'ezdxf.entities.line.Line'> LINE(#63)

In [10]:
#Pen alignment test, no reflection needed

inputfilename = "../dxffiles/Pen-align-test.dxf"
cutlayers, penlayers = ["CUT"], ['PLOT']
outputfilename = os.path.join(d170color, "test.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

layerspresent: {'CUT', 'PLOT'}


[([<class 'ezdxf.entities.line.Line'> LINE(#20015),
   <class 'ezdxf.entities.line.Line'> LINE(#20014),
   <class 'ezdxf.entities.line.Line'> LINE(#20013),
   <class 'ezdxf.entities.line.Line'> LINE(#20016)],
  [False, False, False, False],
  [],
  [<class 'ezdxf.entities.circle.Circle'> CIRCLE(#50),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#51),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#52),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#53),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#54),
   <class 'ezdxf.entities.line.Line'> LINE(#55),
   <class 'ezdxf.entities.line.Line'> LINE(#56)])]